## 1.Understanding Softmax

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


# Cross entropy example
import numpy as np

In [3]:
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print("loss1 = ", np.sum(-Y * np.log(Y_pred1)))
print("loss2 = ", np.sum(-Y * np.log(Y_pred2)))

# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([0]), requires_grad=False)

# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data, "\nPyTorch Loss2=", l2.data)

print("Y_pred1=", torch.max(Y_pred1.data, 1)[1])
print("Y_pred2=", torch.max(Y_pred2.data, 1)[1])

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))


Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 = ", l1.data, "\nBatch Loss2=", l2.data)

loss1 =  0.35667494393873245
loss2 =  2.3025850929940455
PyTorch Loss1 =  tensor(0.4170) 
PyTorch Loss2= tensor(1.8406)
Y_pred1= tensor([0])
Y_pred2= tensor([1])
Batch Loss1 =  tensor(0.4966) 
Batch Loss2= tensor(1.2389)


## Softmax for MNIST

In [4]:
train_dataset = datasets.MNIST(root='./data/mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

Processing...
Done!


In [5]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)



In [7]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [15]:
def train(epoch):
#     model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [13]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.223825
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.022664
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.068304
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.026103
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.081782
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.042719
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.145920
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.114854
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.186010
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.061090
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.024925
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.082258
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.074369
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.168789
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.023616
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.071266
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.132801
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.011901
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.090309
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.063509
Train Epoch: 1 [12800/60000 (

D:\Anaconda-installation\lib\site-packages\ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0016, Accuracy: 9679/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.078548
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.207274
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.100233
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.005579
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.021893
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.010077
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.110096
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.007087
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.014977
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.051976
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.016129
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.035207
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.121192
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.033129
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.063951
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.024323
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.119643
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.026061
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.074900
Train Epoch: 2 [12

Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.019096
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.072681
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.017547
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.049095
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.011702
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.108457
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.026749
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.047422
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.013749
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.059282
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.027734
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.069249
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.054804
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.006184
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.018403
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.074219
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.046896
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.013558
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.099823
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.109475


Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.032865
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.039442
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.056657
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.014609
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.038520
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.051083
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.020096
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.016652
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.031304
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.012959
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.006781
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.027509
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.014156
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.016440
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.068914
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.012684
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.010330
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.016790
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.051741
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.007355


Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.027981
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.056923
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.020681
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.006320
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.009617
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.045042
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.017102
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.029769
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.031949
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.042564
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.021788
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.012708
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.016393
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.018567
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.030722
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.011593
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.002852
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.087639
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.008381
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.047972


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.007121
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.008876
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.020510
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.029225
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.014057
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.025068
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.015851
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.034963
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.094516

Test set: Average loss: 0.0014, Accuracy: 9753/10000 (97%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.014009
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.014608
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.004484
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.024427
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.023010
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.031240
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.005645
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.010100
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.006396
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.037899
Train Epoch: